# Module: Descriptors

1. What is a descriptor?
2. How do they work?
3. How and when to use descriptors?

## What is a descriptor?
An object that has custom behavior when accessed as attrs of other objects.

"They are a general-purpose way of intercepting attribute access." (Datacamp.com)

### The descriptor protocol

```python
__get__(self, obj, type=None) -> object
__set__(self, obj, value) -> None
__delete__(self, obj) -> None
__set_name__(self, owner, name)
```

## How do they work?

In [2]:
class SomeAttr():
    def __get__(self, obj, type=None) -> object:
        print("accessing the attribute to get the value")
        return 'some val'

    def __set__(self, obj, value) -> None:
        print("accessing the attribute to set the value")
        raise AttributeError("Cannot change the value")

class Bar():
    val = SomeAttr()

In [3]:
bar = Bar()

Let's access `val`

**Note** Descriptors are assigned to the class, not the instance.

## How and when to use descriptors?
Descriptors are used under the hood to achieve a lot of Python magic. Let's find some real use cases.

### Exercise
Create a class `Person` with the attributes `age`  and `name` and make it so that if someone tries to instantiate `age` as a negative value, they get an error.

In [1]:
class Person():
    def __init__(self, age, name):
        # Some logic to prevent `age` from being negative
        # otherwise, raise Exception
        self.age = age
        self.name = name

In [2]:
albert = Person(-35, "Albert")

In [14]:
albert = Person(35, "Albert")

What about setting to an invalid value?

We can still re-assign to the invalid value, can't we?

In [3]:
class AgeDescriptor:
    def __init__(self):
        self._age = 0

    def __get__(self, instance, owner):    
        return self._age

    def __set__(self, instance, value):
        if value < 0:
            raise ValueError("Age can never be less than zero")

        self._age = value

    def __delete__(self, instance):
        del self._age

Let's take a look at what's going on here. There are a few salient points:

- `self._age`
- The logic in `__set__`

In [14]:
class Person():
    age = AgeDescriptor()
    
    def __init__(self, age, name):
        self.name = name
        self.age = age

Try to create a `Person` with a negative age.

In [15]:
emily = Person(42, "Emily")

Create a `Person` with positive age.

Then try to change the `age` to be negative.

In [26]:
emily.age = -77

ValueError: Age can never be less than zero

### Exercise
Modify the `Person` class in two key ways:

1. Do not allow `age` to be a string.
2. Do not allow `name` to be anything but a string and it must have a first and last name separated by a whitespace.

Create two `Person` instances and access `age`.

### Let's troubleshoot with a simpler example.
Let's create a Descriptor called `OddNumberValue` that has to be odd.

In [17]:
class OddNumberValue():
    def __init__(self):
        self.value = 0
    def __get__(self, obj, type=None):
        return self.value

    def __set__(self, obj, value):
        if value % 2 == 0:
            raise ValueError("Cant be an even number!")
        self.value = value

In [35]:
class Hello():
    number = OddNumberValue()

Let's instantiate multiple instances and reproduce the problem.

In [36]:
obj_1 = Hello()
obj_2 = Hello()

obj_1.number = 3
print(obj_1.number)
print(obj_2.number)

3
3


And what about another?

### A possible solution
What if we use dictionaries to track the data?

In [40]:
class OddNumberValue2():
    def __init__(self):
        self.value = {}

    def __get__(self, obj, type=None) -> object:
        return self.value.get(obj, 0)

    def __set__(self, obj, value) -> None:
        if value % 2 == 0:
            raise ValueError("Cant be an even number!")
        self.value[obj] = value

In [41]:
class Bye():
    number = OddNumberValue2()

In [43]:
obj_1 = Bye()
obj_2 = Bye()

obj_1.number = 3
obj_2.number = 7
print(obj_1.number)
print(obj_2.number)

3
7


### Discussion
What are some possible issues with the above?

### A better way

In [44]:
obj_2.__dict__

{}

In [53]:
class OddNumberFinal():
    def __init__(self, name):
        self.name = name

    def __get__(self, obj, type=None) -> object:
        return obj.__dict__.get(self.name) or 0

    def __set__(self, obj, value) -> None:
        # Add your odd number logic here
        if value % 2 == 0:
            raise AttributeError("cant be even!")
        obj.__dict__[self.name] = value

In [54]:
class Yo():
    number = OddNumberFinal("number")

In [55]:
obj_1 = Yo()
obj_2 = Yo()

obj_1.number = 3
print(obj_1.number)
print(obj_2.number)

3
0


### Let's introspect a bit.
What is in `obj_1.__dict__`

In [56]:
obj_1.__dict__

{'number': 3}

Let's try to set it to an even number.

### Homework Exercise
Use `.__set_name__` to avoid having to pass in the name of the instance attribute

**Note**: Feel free to Google.

## (Optional) Lab: Descriptors
Update `Person` to work for multiple instances, similar to how we fixed the `OddNumberValue` descriptor.